In [1]:
# Import Modules 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from mmdata import *

In [2]:
team_resume

,YEAR,TEAM NO,TEAM,SEED,ROUND,NET RPI,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q2 W,Q1 PLUS Q2 W,Q3 Q4 L,PLUS 500,R SCORE,BID TYPE
0,2024,1079,Akron,14,0,106,131,104,103,111.5,0,2,2,4,13,0.00,At-Large
1,2024,1078,Alabama,4,0,10,31,15,31,11.5,4,7,11,0,11,99.35,At-Large
2,2024,1077,Arizona,2,0,4,10,11,14,5.5,8,7,15,1,18,99.87,At-Large
3,2024,1076,Auburn,4,0,5,45,9,5,4.0,3,10,13,0,21,99.42,At-Large
4,2024,1075,Baylor,3,0,13,8,10,25,13.5,10,4,14,0,14,99.73,At-Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,2008,5,West Virginia,7,16,29,32,28,29,20.0,3,2,5,1,13,96.80,At-Large
1075,2008,4,Western Kentucky,12,16,39,124,46,25,45.0,0,1,1,1,19,5.30,Auto
1076,2008,3,Winthrop,13,64,108,121,108,93,96.0,1,2,3,7,9,0.00,Auto
1077,2008,2,Wisconsin,3,16,11,9,8,5,6.0,6,2,8,0,25,99.90,Auto


In [3]:
# Clean ratings data
ratings = ratings.groupby(['YEAR', 'TEAM']).sum().reset_index()
ratings

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK
0,2016,Arizona,603,6,64,89.0,59
1,2016,Austin Peay,602,16,64,68.8,462
2,2016,Baylor,601,5,64,85.5,152
3,2016,Buffalo,600,14,64,75.7,396
4,2016,Butler,599,9,32,84.2,194
...,...,...,...,...,...,...,...
471,2023,VCU,948,12,64,81.7,278
472,2023,Vermont,947,15,64,76.3,386
473,2023,Virginia,946,4,64,83.8,205
474,2023,West Virginia,945,9,64,84.2,194


In [4]:
# Clean team_resume
team_resume = team_resume.drop(columns=['BID TYPE'])
team_resume = team_resume.groupby(['YEAR', 'TEAM']).sum().reset_index()
team_resume

,YEAR,TEAM,TEAM NO,SEED,ROUND,NET RPI,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q2 W,Q1 PLUS Q2 W,Q3 Q4 L,PLUS 500,R SCORE
0,2008,American,65,15,64,91,183,135,98,149.0,0,1,1,8,10,0.00
1,2008,Arizona,64,10,64,37,16,36,59,19.0,5,5,10,2,4,91.80
2,2008,Arkansas,63,9,32,31,15,43,42,42.0,6,3,9,4,11,95.00
3,2008,Austin Peay,62,15,64,84,165,127,100,163.0,0,2,2,5,14,0.00
4,2008,BYU,57,8,64,25,42,29,19,34.0,2,6,8,0,20,93.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,2024,Wagner,1016,16,0,290,304,241,253,283.0,0,0,0,13,1,0.00
1075,2024,Washington St.,1015,7,0,45,28,29,32,42.0,6,3,9,1,15,86.43
1076,2024,Western Kentucky,1014,15,0,136,204,135,130,145.0,0,1,1,11,10,0.00
1077,2024,Wisconsin,1013,5,0,18,18,18,45,17.5,5,9,14,0,9,98.88


In [5]:
# Merge ratings and team_resume
final_data1 = pd.merge(ratings, team_resume, on=['YEAR', 'TEAM'])
final_data1

,YEAR,TEAM,TEAM NO_x,SEED_x,ROUND_x,POWER RATING,POWER RATING RANK,TEAM NO_y,SEED_y,ROUND_y,...,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q2 W,Q1 PLUS Q2 W,Q3 Q4 L,PLUS 500,R SCORE
0,2016,Arizona,603,6,64,89.0,59,603,6,64,...,27,23,20,16.0,4,9,13,1,17,97.7
1,2016,Austin Peay,602,16,64,68.8,462,602,16,64,...,218,229,175,219.0,0,1,1,10,-1,0.0
2,2016,Baylor,601,5,64,85.5,152,601,5,64,...,25,22,40,24.0,5,5,10,0,10,97.9
3,2016,Buffalo,600,14,64,75.7,396,600,14,64,...,112,129,108,141.0,1,1,2,5,5,0.0
4,2016,Butler,599,9,32,84.2,194,599,9,32,...,29,32,37,39.0,4,2,6,1,11,76.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2023,VCU,948,12,64,81.7,278,948,12,64,...,89,44,22,59.0,1,4,5,2,20,5.8
472,2023,Vermont,947,15,64,76.3,386,947,15,64,...,137,109,71,104.0,0,1,1,4,12,0.0
473,2023,Virginia,946,4,64,83.8,205,946,4,64,...,33,18,21,28.3,5,6,11,1,18,99.2
474,2023,West Virginia,945,9,64,84.2,194,945,9,64,...,25,36,50,20.3,6,6,12,0,5,93.3


In [6]:
# Clean final_data1
final_data1 = final_data1.drop(columns=['TEAM NO_y', 'SEED_y', 'ROUND_y'])
final_data1 = final_data1.rename(columns={'TEAM NO_x': 'TEAM NO', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})
final_data1

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q2 W,Q1 PLUS Q2 W,Q3 Q4 L,PLUS 500,R SCORE
0,2016,Arizona,603,6,64,89.0,59,26,27,23,20,16.0,4,9,13,1,17,97.7
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,175,219.0,0,1,1,10,-1,0.0
2,2016,Baylor,601,5,64,85.5,152,25,25,22,40,24.0,5,5,10,0,10,97.9
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,108,141.0,1,1,2,5,5,0.0
4,2016,Butler,599,9,32,84.2,194,56,29,32,37,39.0,4,2,6,1,11,76.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2023,VCU,948,12,64,81.7,278,58,89,44,22,59.0,1,4,5,2,20,5.8
472,2023,Vermont,947,15,64,76.3,386,109,137,109,71,104.0,0,1,1,4,12,0.0
473,2023,Virginia,946,4,64,83.8,205,26,33,18,21,28.3,5,6,11,1,18,99.2
474,2023,West Virginia,945,9,64,84.2,194,24,25,36,50,20.3,6,6,12,0,5,93.3


In [7]:
#Clean tournament_matchups 
tournament_matchups = tournament_matchups.fillna(0)
tournament_matchups

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE
0,2024,2036,0,1067,Connecticut,1,1,64,0.0
1,2024,2035,0,1026,Stetson,16,64,64,0.0
2,2024,2034,0,1038,North Carolina,1,1,64,0.0
3,2024,2033,0,1055,Howard,16,64,64,0.0
4,2024,2032,0,1038,North Carolina,1,1,64,0.0
...,...,...,...,...,...,...,...,...,...
2031,2008,5,33,43,Kansas,1,1,4,84.0
2032,2008,4,32,37,Memphis,1,2,4,78.0
2033,2008,3,31,13,UCLA,1,4,4,63.0
2034,2008,2,2,43,Kansas,1,1,2,75.0


In [8]:
# Merge final_data1 and tournament_matchups
final_data2 = pd.merge(final_data1, tournament_matchups, on=['YEAR', 'TEAM NO'])
final_data2

,YEAR,TEAM_x,TEAM NO,SEED_x,ROUND_x,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,Q3 Q4 L,PLUS 500,R SCORE,BY YEAR NO,BY ROUND NO,TEAM_y,SEED_y,ROUND_y,CURRENT ROUND,SCORE
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,1,17,97.7,1126,1498,Arizona,6,64,64,55.0
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,...,10,-1,0.0,1133,1505,Austin Peay,16,64,64,79.0
2,2016,Baylor,601,5,64,85.5,152,25,25,22,...,0,10,97.9,1114,1486,Baylor,5,64,64,75.0
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,...,5,5,0.0,1123,1495,Buffalo,14,64,64,72.0
4,2016,Butler,599,9,32,84.2,194,56,29,32,...,1,11,76.7,1083,1455,Butler,9,32,64,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023,Virginia,946,4,64,83.8,205,26,33,18,...,1,18,99.2,1882,1882,Virginia,4,64,64,67.0
871,2023,West Virginia,945,9,64,84.2,194,24,25,36,...,0,5,93.3,1885,1885,West Virginia,9,64,64,65.0
872,2023,Xavier,944,3,16,84.4,186,22,11,11,...,2,16,99.5,1846,1846,Xavier,3,16,64,72.0
873,2023,Xavier,944,3,16,84.4,186,22,11,11,...,2,16,99.5,1803,909,Xavier,3,16,32,84.0


In [9]:
#Clean final_data2
final_data2 = final_data2.rename(columns={'TEAM_x': 'TEAM', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})
final_data2 = final_data2.drop(columns=['TEAM_y', 'SEED_y', 'ROUND_y'])
final_data2

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,Q1 W,Q2 W,Q1 PLUS Q2 W,Q3 Q4 L,PLUS 500,R SCORE,BY YEAR NO,BY ROUND NO,CURRENT ROUND,SCORE
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,4,9,13,1,17,97.7,1126,1498,64,55.0
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,...,0,1,1,10,-1,0.0,1133,1505,64,79.0
2,2016,Baylor,601,5,64,85.5,152,25,25,22,...,5,5,10,0,10,97.9,1114,1486,64,75.0
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,...,1,1,2,5,5,0.0,1123,1495,64,72.0
4,2016,Butler,599,9,32,84.2,194,56,29,32,...,4,2,6,1,11,76.7,1083,1455,64,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023,Virginia,946,4,64,83.8,205,26,33,18,...,5,6,11,1,18,99.2,1882,1882,64,67.0
871,2023,West Virginia,945,9,64,84.2,194,24,25,36,...,6,6,12,0,5,93.3,1885,1885,64,65.0
872,2023,Xavier,944,3,16,84.4,186,22,11,11,...,8,5,13,2,16,99.5,1846,1846,64,72.0
873,2023,Xavier,944,3,16,84.4,186,22,11,11,...,8,5,13,2,16,99.5,1803,909,32,84.0


In [10]:
#Clean team_results
team_results = team_results.drop(columns=['TEAM ID'])
team_results = team_results.groupby(['TEAM']).sum().reset_index()
team_results

,TEAM,PAKE,PAKE RANK,PASE,PASE RANK,GAMES,W,L,WIN%,R64,R32,S16,E8,F4,F2,CHAMP,TOP2,F4%,CHAMP%
0,Abilene Christian,0.7,45,0.7,52,3,1,2,0.333,2,1,0,0,0,0,0,0,0.10%,0.00%
1,Akron,-0.9,179,-1.1,187,4,0,4,0.000,4,0,0,0,0,0,0,0,0.50%,0.00%
2,Alabama,-2.1,211,-2.9,220,10,5,5,0.500,5,3,2,0,0,0,0,2,48.40%,15.40%
3,Albany,-0.4,147,-0.3,138,3,0,3,0.000,3,0,0,0,0,0,0,0,0.00%,0.00%
4,American,-0.5,160,-0.4,150,3,0,3,0.000,3,0,0,0,0,0,0,0,0.20%,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,Wofford,-1.2,191,-0.9,175,6,1,5,0.167,5,1,0,0,0,0,0,0,3.60%,0.20%
232,Wright St.,-0.2,118,-0.2,119,2,0,2,0.000,2,0,0,0,0,0,0,0,0.00%,0.00%
233,Wyoming,-0.2,118,-0.5,159,1,0,1,0.000,1,0,0,0,0,0,0,0,0.00%,0.00%
234,Xavier,4.1,14,2.7,22,28,18,10,0.643,10,9,7,2,0,0,0,2,51.90%,8.70%


In [11]:
# Merge into final
final_data3 = pd.merge(final_data2, team_results, on=['TEAM'])
final_data3

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,R64,R32,S16,E8,F4,F2,CHAMP,TOP2,F4%,CHAMP%
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,11,7,7,3,0,0,0,5,83.50%,36.10%
1,2017,Arizona,671,2,16,89.0,59,2,15,6,...,11,7,7,3,0,0,0,5,83.50%,36.10%
2,2017,Arizona,671,2,16,89.0,59,2,15,6,...,11,7,7,3,0,0,0,5,83.50%,36.10%
3,2017,Arizona,671,2,16,89.0,59,2,15,6,...,11,7,7,3,0,0,0,5,83.50%,36.10%
4,2018,Arizona,738,4,64,87.9,84,13,32,19,...,11,7,7,3,0,0,0,5,83.50%,36.10%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023,Kennesaw St.,984,14,64,73.3,422,114,136,69,...,1,0,0,0,0,0,0,0,0.10%,0.00%
871,2023,Louisiana Lafayette,981,13,64,76.3,386,89,129,61,...,2,0,0,0,0,0,0,0,0.30%,0.00%
872,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,2,1,0,0,0,0,0,0,3.80%,0.30%
873,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,2,1,0,0,0,0,0,0,3.80%,0.30%


In [12]:
final_data3.columns

Index(['YEAR', 'TEAM', 'TEAM NO', 'SEED', 'ROUND', 'POWER RATING',
       'POWER RATING RANK', 'NET RPI', 'RESUME', 'WAB RANK', 'ELO', 'B POWER',
       'Q1 W', 'Q2 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'PLUS 500', 'R SCORE',
       'BY YEAR NO', 'BY ROUND NO', 'CURRENT ROUND', 'SCORE', 'PAKE',
       'PAKE RANK', 'PASE', 'PASE RANK', 'GAMES', 'W', 'L', 'WIN%', 'R64',
       'R32', 'S16', 'E8', 'F4', 'F2', 'CHAMP', 'TOP2', 'F4%', 'CHAMP%'],
      dtype='object')

In [13]:
#Clean heat check
heat_check = heat_check.fillna(0)
heat_check = heat_check.groupby(['YEAR', 'TEAM']).sum().reset_index()
heat_check

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER,PATH,DRAW,WINS,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH
0,2013,Akron,399,12,64,65.3,74.2,0.31,0,7.3,40,49,1.004,-8.9
1,2013,Albany,398,15,64,50.7,84.7,-0.59,0,1.0,61,60,-0.749,-34.0
2,2013,Arizona,397,6,16,77.6,69.9,0.34,2,33.8,11,21,1.696,7.7
3,2013,Belmont,396,11,64,65.6,76.6,-0.80,0,6.6,41,44,-0.339,-11.0
4,2013,Bucknell,394,11,64,64.9,69.1,2.02,0,12.7,27,47,0.654,-4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,2024,Wagner,1016,16,0,44.7,77.8,0.00,0,1.1,59,64,0.000,-33.1
700,2024,Washington St.,1015,7,0,66.4,68.8,0.00,0,12.0,24,26,0.000,-2.4
701,2024,Western Kentucky,1014,15,0,52.5,80.0,0.00,0,1.6,58,58,0.000,-27.5
702,2024,Wisconsin,1013,5,0,68.9,71.7,0.00,0,9.1,27,19,0.000,-2.8


In [14]:
# Merge into final
final_data4 = pd.merge(final_data3, heat_check, on=['YEAR', 'TEAM'])
final_data4

,YEAR,TEAM,TEAM NO_x,SEED_x,ROUND_x,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,ROUND_y,POWER,PATH,DRAW,WINS,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,64,80.4,76.0,-1.53,0,27.5,17,23,1.018,4.4
1,2017,Arizona,671,2,16,89.0,59,2,15,6,...,16,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
2,2017,Arizona,671,2,16,89.0,59,2,15,6,...,16,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
3,2017,Arizona,671,2,16,89.0,59,2,15,6,...,16,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
4,2018,Arizona,738,4,64,87.9,84,13,32,19,...,64,80.7,69.5,-0.89,0,41.6,10,16,0.212,11.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023,Kennesaw St.,984,14,64,73.3,422,114,136,69,...,64,51.8,73.4,1.28,0,2.6,54,55,-0.117,-21.6
871,2023,Louisiana Lafayette,981,13,64,76.3,386,89,129,61,...,64,56.9,81.5,-1.01,0,1.8,58,54,-0.958,-24.6
872,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,32,63.5,67.8,1.67,1,7.7,37,38,-0.038,-4.3
873,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,32,63.5,67.8,1.67,1,7.7,37,38,-0.038,-4.3


In [15]:
final_data4.columns

Index(['YEAR', 'TEAM', 'TEAM NO_x', 'SEED_x', 'ROUND_x', 'POWER RATING',
       'POWER RATING RANK', 'NET RPI', 'RESUME', 'WAB RANK', 'ELO', 'B POWER',
       'Q1 W', 'Q2 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'PLUS 500', 'R SCORE',
       'BY YEAR NO', 'BY ROUND NO', 'CURRENT ROUND', 'SCORE', 'PAKE',
       'PAKE RANK', 'PASE', 'PASE RANK', 'GAMES', 'W', 'L', 'WIN%', 'R64',
       'R32', 'S16', 'E8', 'F4', 'F2', 'CHAMP', 'TOP2', 'F4%', 'CHAMP%',
       'TEAM NO_y', 'SEED_y', 'ROUND_y', 'POWER', 'PATH', 'DRAW', 'WINS',
       'POOL VALUE', 'POOL S-RANK', 'NCAA S-RANK', 'VAL Z-SCORE',
       'POWER-PATH'],
      dtype='object')

In [16]:
#Clean final_data4
final_data4 = final_data4.drop(columns=['TEAM NO_y', 'SEED_y', 'ROUND_y'])
final_data4 = final_data4.rename(columns={'TEAM NO_x': 'TEAM NO', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})
final_data4

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,CHAMP%,POWER,PATH,DRAW,WINS,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,36.10%,80.4,76.0,-1.53,0,27.5,17,23,1.018,4.4
1,2017,Arizona,671,2,16,89.0,59,2,15,6,...,36.10%,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
2,2017,Arizona,671,2,16,89.0,59,2,15,6,...,36.10%,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
3,2017,Arizona,671,2,16,89.0,59,2,15,6,...,36.10%,83.6,66.9,-0.37,2,44.7,12,6,-0.783,16.7
4,2018,Arizona,738,4,64,87.9,84,13,32,19,...,36.10%,80.7,69.5,-0.89,0,41.6,10,16,0.212,11.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023,Kennesaw St.,984,14,64,73.3,422,114,136,69,...,0.00%,51.8,73.4,1.28,0,2.6,54,55,-0.117,-21.6
871,2023,Louisiana Lafayette,981,13,64,76.3,386,89,129,61,...,0.00%,56.9,81.5,-1.01,0,1.8,58,54,-0.958,-24.6
872,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,0.30%,63.5,67.8,1.67,1,7.7,37,38,-0.038,-4.3
873,2023,Penn St.,967,10,32,82.8,240,49,21,38,...,0.30%,63.5,67.8,1.67,1,7.7,37,38,-0.038,-4.3


In [17]:
#Upsets
upset_seed_info = upeset_seed_info.groupby(['YEAR', 'CURRENT ROUND']).sum().reset_index()
upset_seed_info.head()

,YEAR,CURRENT ROUND,SEED WON,SEED LOST,SEED DIFF
0,2008,16,10,3,7
1,2008,32,17,4,13
2,2008,64,71,31,40
3,2009,8,3,1,2
4,2009,64,90,46,44


In [18]:
# Merge into final
final_data5 = pd.merge(final_data4, upset_seed_info, on=['YEAR', 'CURRENT ROUND'])
final_data5

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,DRAW,WINS,POOL VALUE,POOL S-RANK,NCAA S-RANK,VAL Z-SCORE,POWER-PATH,SEED WON,SEED LOST,SEED DIFF
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,-1.53,0,27.5,17,23,1.018,4.4,119,51,68
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,...,-0.38,0,0.5,64,61,-1.153,-41.8,119,51,68
2,2016,Baylor,601,5,64,85.5,152,25,25,22,...,-0.27,0,27.2,18,20,0.285,5.1,119,51,68
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,...,0.39,0,2.0,56,54,-0.554,-24.4,119,51,68
4,2016,Butler,599,9,32,84.2,194,56,29,32,...,1.43,1,10.5,30,34,0.645,-3.5,119,51,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,2016,Oklahoma,567,2,4,90.0,44,6,3,5,...,0.96,4,91.9,2,6,1.196,19.4,10,1,9
780,2016,Oregon,566,1,8,88.0,81,2,4,10,...,1.23,3,33.4,14,4,-1.820,11.7,10,1,9
781,2016,Syracuse,555,10,4,82.7,244,72,32,59,...,0.68,4,9.7,32,39,0.488,-3.4,10,1,9
782,2016,Villanova,543,2,1,91.3,26,4,9,2,...,-0.11,6,51.4,7,7,-0.502,15.4,10,1,9


In [19]:
final_data5.columns

Index(['YEAR', 'TEAM', 'TEAM NO', 'SEED', 'ROUND', 'POWER RATING',
       'POWER RATING RANK', 'NET RPI', 'RESUME', 'WAB RANK', 'ELO', 'B POWER',
       'Q1 W', 'Q2 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'PLUS 500', 'R SCORE',
       'BY YEAR NO', 'BY ROUND NO', 'CURRENT ROUND', 'SCORE', 'PAKE',
       'PAKE RANK', 'PASE', 'PASE RANK', 'GAMES', 'W', 'L', 'WIN%', 'R64',
       'R32', 'S16', 'E8', 'F4', 'F2', 'CHAMP', 'TOP2', 'F4%', 'CHAMP%',
       'POWER', 'PATH', 'DRAW', 'WINS', 'POOL VALUE', 'POOL S-RANK',
       'NCAA S-RANK', 'VAL Z-SCORE', 'POWER-PATH', 'SEED WON', 'SEED LOST',
       'SEED DIFF'],
      dtype='object')

In [20]:
#Num of upsets
num_of_upsets = num_of_upsets.groupby(['YEAR']).sum().reset_index()
num_of_upsets

,YEAR,FIRST ROUND,SECOND ROUND,SWEET 16,ELITE 8,FINAL 4,TOTAL
0,2008,6,2,1,0,0,9
1,2009,8,0,0,1,0,9
2,2010,8,5,2,1,0,16
3,2011,6,4,3,3,0,16
4,2012,9,2,2,0,0,13
5,2013,8,3,2,1,0,14
6,2014,7,5,3,2,2,19
7,2015,5,3,1,1,0,10
8,2016,10,2,0,1,0,13
9,2017,5,4,2,2,0,13


In [21]:
# Merge into final
final_data = pd.merge(final_data5, num_of_upsets, on=['YEAR'])
final_data

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,POWER-PATH,SEED WON,SEED LOST,SEED DIFF,FIRST ROUND,SECOND ROUND,SWEET 16,ELITE 8,FINAL 4,TOTAL
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,4.4,119,51,68,10,2,0,1,0,13
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,...,-41.8,119,51,68,10,2,0,1,0,13
2,2016,Baylor,601,5,64,85.5,152,25,25,22,...,5.1,119,51,68,10,2,0,1,0,13
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,...,-24.4,119,51,68,10,2,0,1,0,13
4,2016,Butler,599,9,32,84.2,194,56,29,32,...,-3.5,119,51,68,10,2,0,1,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,2021,Oregon St.,830,12,8,79.7,332,91,32,47,...,-19.5,11,1,10,9,6,2,1,0,18
780,2021,USC,817,6,8,85.8,143,19,37,7,...,7.5,11,1,10,9,6,2,1,0,18
781,2021,Arkansas,872,3,8,86.3,129,14,20,12,...,7.6,11,1,10,9,6,2,1,0,18
782,2021,UCLA,819,11,4,81.9,267,46,55,24,...,-6.8,11,1,10,9,6,2,1,0,18


In [23]:
# Save final data
final_data.to_csv('final_data.csv', index=False)
final_data.columns

Index(['YEAR', 'TEAM', 'TEAM NO', 'SEED', 'ROUND', 'POWER RATING',
       'POWER RATING RANK', 'NET RPI', 'RESUME', 'WAB RANK', 'ELO', 'B POWER',
       'Q1 W', 'Q2 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'PLUS 500', 'R SCORE',
       'BY YEAR NO', 'BY ROUND NO', 'CURRENT ROUND', 'SCORE', 'PAKE',
       'PAKE RANK', 'PASE', 'PASE RANK', 'GAMES', 'W', 'L', 'WIN%', 'R64',
       'R32', 'S16', 'E8', 'F4', 'F2', 'CHAMP', 'TOP2', 'F4%', 'CHAMP%',
       'POWER', 'PATH', 'DRAW', 'WINS', 'POOL VALUE', 'POOL S-RANK',
       'NCAA S-RANK', 'VAL Z-SCORE', 'POWER-PATH', 'SEED WON', 'SEED LOST',
       'SEED DIFF', 'FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8',
       'FINAL 4', 'TOTAL'],
      dtype='object')